## Using the input template excel file template to import the variables and objectives data

In [ ]:
# Import the variable, objectives, sampler, acquisition function, and the optimisation classes
from typing import Dict, Tuple

import numpy as np

from nemo_bo.utils.extract_excel import extract_input_template
from nemo_bo.opt.objectives import DeterministicFunction
from nemo_bo.acquisition_functions.expected_improvement.expected_improvement import (
    ExpectedImprovement,
)
from nemo_bo.opt.samplers import LatinHyperCubeSampling
from nemo_bo.opt.optimisation import Optimisation

### Using the input template excel file

The excel file will always contain the sheets "Variables" and "Objectives", and additional sheets for every categorical variable with descriptors where the name of the sheet is the same as the name of the categorical variable.

Inside the "Variables" and "Objectives" sheets, the same type of information as the arguments and some keyword arguments normally required can be provided. The cells for the keyword arguments can be left blank in the template. The variables (X) and objectives (Y) values are also in the respective sheets.

The categorical variable with descriptors sheet contains the name of the categorical levels and their corresponding descriptor names and descriptor values

For calculable objectives, the function (an instance of a `DeterministicFunction`) to calculate the target value using the variables needs to be added afterwards using the `add_calculable_obj_function` method. See the `defining_calculable_objectives.ipynb` tutorial for how to use the `DeterministicFunction` class.

In [ ]:
# Import the variables and objectives data using the input template excel file (input_template.xlsx)
X, Y, var_list, obj_list = extract_input_template("extract_excel_example_template.xlsx")

# Define the deterministic function for the calculable objective
# Cost calculation used for this example
class Cost_calculation(DeterministicFunction):
    def __init__(self, obj_function_data: Dict):
        super().__init__(obj_function_data)

    def evaluate(self, X: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        cost_array = np.zeros(X.shape[0])
        for index, x in enumerate(X):
            cost_array[index] = obj_function_data[x[1]]

        return cost_array, np.zeros_like(cost_array)


# Instantiate the deterministic function and pass in the any required calculation information
obj_function_data = {
    "cat_level1": 2.05,
    "cat_level2": 1.00,
    "cat_level3": 1.50,
    "cat_level4": 5.00,
}
det_func = Cost_calculation(obj_function_data)

# Add the calculable objective's function
obj_list.add_calculable_obj_function(name="Cost", obj_function=det_func)

In [ ]:
# Instantiate the sampler
sampler = LatinHyperCubeSampling()

In [ ]:
# Instantiate the acquisition function
acq_func = ExpectedImprovement(num_candidates=4)

In [ ]:
# Set up the optimisation instance
optimisation = Optimisation(var_list, obj_list, acq_func, sampler=sampler)

In [ ]:
# Start the optimisation using the convenient run function that will run for the specified number of iterations
optimisation_data = optimisation.run(X, Y, number_of_iterations=50)